<a href="https://colab.research.google.com/github/MiM0ulay/RiskMetrics/blob/main/SOLUSDRiskMetricFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install nasdaq-data-link


In [2]:
from datetime import date
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
import nasdaqdatalink
import yfinance as yf

In [3]:

# Download data
df = yf.download(tickers='SOL-USD', start='2020-01-01', interval='1d')

# Debugging: Check the column names
print("Column names before processing:", df.columns)

# Reset the index and flatten column names (if needed)
df.reset_index(inplace=True)

# Handle multi-level columns (flatten them if they exist)
df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]

# Debugging: Check flattened column names
print("Column names after flattening:", df.columns)

# Rename columns for consistency
if 'Open' in df.columns:
    df.rename(columns={'Date': 'date', 'Open': 'value'}, inplace=True)
elif 'value' not in df.columns:
    raise KeyError("The required columns ('date', 'value') are not found in the DataFrame.")

# Keep necessary columns
df = df[['date', 'value']].copy()

# Sort by date
df.sort_values(by='date', inplace=True)

# Calculate moving average
moving_average_days = 365
df['MA'] = df['value'].rolling(moving_average_days, min_periods=1).mean()

# Ensure no NaN values
df = df.dropna().reset_index(drop=True)

# Calculate Preavg
diminishing_factor = 0.395
df['Preavg'] = (np.log(df['value']) - np.log(df['MA'])) * (np.arange(len(df)) + 1) ** diminishing_factor

# Print results
print(df.head())


[*********************100%***********************]  1 of 1 completed

Column names before processing: MultiIndex([('Adj Close', 'SOL-USD'),
            (    'Close', 'SOL-USD'),
            (     'High', 'SOL-USD'),
            (      'Low', 'SOL-USD'),
            (     'Open', 'SOL-USD'),
            (   'Volume', 'SOL-USD')],
           names=['Price', 'Ticker'])
Column names after flattening: Index(['Date', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume'], dtype='object')
        date     value        MA    Preavg
0 2020-04-10  0.832005  0.832005  0.000000
1 2020-04-11  0.951054  0.891530  0.084988
2 2020-04-12  0.785448  0.856169 -0.133057
3 2020-04-13  0.890760  0.864817  0.051107
4 2020-04-14  0.777832  0.847420 -0.161809


In [4]:
print("Column names:", df.columns)


Column names: Index(['date', 'value', 'MA', 'Preavg'], dtype='object')


In [5]:
# Normalization to 0-1 range
df['avg'] = (df['Preavg'] - df['Preavg'].cummin()) / (df['Preavg'].cummax() - df['Preavg'].cummin())

In [6]:
price_per_risk = {
    round(risk, 1):round(np.exp(
        (risk * (df['Preavg'].cummax().iloc[-1] - (cummin := df['Preavg'].cummin().iloc[-1])) + cummin) / df.index[-1]**diminishing_factor + np.log(df['MA'].iloc[-1])
    ))
    for risk in np.arange(0.0, 1.0, 0.1)
}

In [7]:
# # Exclude the first 1000 days from the dataframe, because it's pure chaos
AnnotationText = f"Updated: {df.index[-1]} | Price: {round(df['value'].iloc[-1])} | Risk: {round(df['avg'].iloc[-1], 2)}"

In [8]:
# Plot BTC-USD and Risk on a logarithmic chart
fig = make_subplots(specs=[[{'secondary_y': True}]])

# Add BTC-USD and Risk data to the figure
fig.add_trace(go.Scatter(x=df['date'], y=df['value'], name='Price', line=dict(color='gold')))
fig.add_trace(go.Scatter(x=df['date'], y=df['avg'],   name='Risk',  line=dict(color='white')), secondary_y=True)

# Add green (`accumulation` or `buy`) rectangles to the figure
opacity = 0.2
for i in range(5, 0, -1):
    opacity += 0.05
    fig.add_hrect(y0=i*0.1, y1=((i-1)*0.1), line_width=0, fillcolor='green', opacity=opacity, secondary_y=True)

# Add red (`distribution` or `sell`) rectangles to the figure
opacity = 0.2
for i in range(6, 10):
    opacity += 0.1
    fig.add_hrect(y0=i*0.1, y1=((i+1)*0.1), line_width=0, fillcolor='red', opacity=opacity, secondary_y=True)

fig.update_xaxes(title='Date')
fig.update_yaxes(title='Price ($USD)', type='log', showgrid=False)
fig.update_yaxes(title='Risk', type='linear', secondary_y=True, showgrid=True, tick0=0.0, dtick=0.1, range=[0, 1])
fig.update_layout(template='plotly_dark', title={'text': AnnotationText, 'y': 0.9, 'x': 0.5})
fig.show()

# Plot BTC-USD colored according to Risk values on a logarithmic chart
fig = px.scatter(df, x='date', y='value', color='avg', color_continuous_scale='jet')
fig.update_yaxes(title='Price ($USD)', type='log', showgrid=False)
fig.update_layout(template='plotly_dark', title={'text': AnnotationText, 'y': 0.9, 'x': 0.5})
fig.show()

# Plot Predicting BTC price according to specific risk
fig = go.Figure(data=[go.Table(
    header=dict(values=['Risk', 'Price'],
                line_color='darkslategray',
                fill_color='lightskyblue',
                align='left'),
    cells=dict(values=[list(price_per_risk.keys()), list(price_per_risk.values())],
               line_color='darkslategray',
               fill_color='lightcyan',
               align='left'))
])
fig.update_layout(width=500, height=500, title={'text': 'Price according to specific risk', 'y': 0.9, 'x': 0.5})
fig.show()

In [9]:
df.tail(60)

,date,value,MA,Preavg,avg
1663,2024-10-29,178.103638,130.219977,5.862661,0.627208
1664,2024-10-30,179.360748,130.615596,5.938959,0.628612
1665,2024-10-31,174.925079,130.989327,5.417799,0.619026
1666,2024-11-01,168.430222,131.337394,4.660485,0.605096
1667,2024-11-02,166.262131,131.682745,4.369585,0.599745
1668,2024-11-03,166.010345,132.029293,4.292948,0.598335
1669,2024-11-04,162.551697,132.358549,3.852529,0.590234
1670,2024-11-05,157.747101,132.677751,3.245616,0.579071
1671,2024-11-06,166.722397,133.019078,4.236181,0.597291
1672,2024-11-07,186.954788,133.413240,6.330614,0.635816
